In [41]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

data = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
t_data = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
test_data = t_data.iloc[:,1:15]
row_id = t_data.iloc[:,0]
X = data.iloc[:,1:15]
X_copy = X
Y = data.iloc[:,15]

In [42]:
columns = []
for col in X.columns:
    columns.append(col)
print(columns)


['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14']


In [43]:
for i in columns:
    for j in columns[columns.index(i)+1:]:
        X[i+'_'+j] = X[i]+X[j]
        test_data[i+'_'+j]= test_data[i]+test_data[j]

In [30]:

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
sgdreg = SGDRegressor(max_iter = 1000, tol = 1e-3, penalty = 'l1', eta0 = 0.1)

sfs = SFS(LinearRegression(), k_features = 20, forward = True, floating = False, scoring = 'r2', cv =0)
sfs.fit(X,Y)
top_features = sfs.k_feature_names_


In [44]:
use_data = X.filter(top_features)
test_use_data = test_data.filter(top_features)
test_use_data.shape

(200000, 20)

In [32]:
from sklearn.model_selection import train_test_split

data['target'].describe()

bins = [0,5,6,7,8,9,10,11]
y_binned = np.digitize(Y,bins)
X_train,X_val,Y_train,Y_val = train_test_split(use_data,Y, test_size = 0.3, random_state = 42, stratify = y_binned)

In [8]:
'''
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error
linreg = LinearRegression()
sgdreg = SGDRegressor(tol = 1e-4, max_iter = 1000, penalty = 'l2')
linreg.fit(X_train,Y_train)
y_pred = linreg.predict(X_val)
score = mean_squared_error(y_pred,Y_val)
print('score: ', score)


score:  0.5263072307262087


In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

gbrt = GradientBoostingRegressor(max_depth = 5, warm_start = True)
min_score = np.infty
cnt = 0
buffer = 0
for n_estimators in range(1,100):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train,Y_train)
    y_pred = gbrt.predict(X_val)
    score = mean_squared_error(y_pred,Y_val)
    
    if score<min_score:
        min_score = score
        best_cnt = n_estimators
        buffer_time = 0
    else:
        buffer+=1
        if buffer>3:
            break
print('best score: ', min_score)
print('Optimal Estimators: ', best_cnt)


best score:  0.5087072200833646
Optimal Estimators:  99


In [45]:
predictions = pd.DataFrame(gbrt.predict(test_use_data))
predictions['id'] = row_id
predictions.set_axis(['target','id'], axis =1, inplace = True)
predictions = predictions[['id','target']]
predictions.to_csv('./submission.csv', index = False)